In [2]:
!pip3 install -U -q sagemaker boto3 langchain streamlit ipykernel "ai21[AWS]"

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.
awscli 1.29.14 requires botocore==1.31.14, but you have botocore 1.31.20 which is incompatible.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.3.2 which is incompatible.
jupyterlab 3.4.4 requires jupyter-server~=1.16, but you have jupyter-server 2.7.0 which is incompatible.
notebook 6.5.5 requires jupyter-client<8,>=5.3.4, but you have jupyter-client 8.3.0 which is incompatible.
notebook 6.5.5 requires pyzmq<25,>=17, but you have pyzmq 25.1.0 which is incompatible.
panel 0.13.1 requires bokeh<2.5.0,>=2.4.0, but you have bokeh 3.2.1 which is incompatible.
sagemaker-datawrangler 0.4.3 requires sagemaker-data-insights==0.4.0, but you have sagemaker-data-insights 0.3.3 

In [3]:
import sagemaker
import boto3
import ai21
import os
from sagemaker import ModelPackage
import json as json
import ai21 

from langchain.retrievers import AmazonKendraRetriever
from langchain import SagemakerEndpoint
from langchain.chains import RetrievalQA
from langchain.llms.sagemaker_endpoint import LLMContentHandler
from langchain.prompts import PromptTemplate




/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
boto3_session = boto3.session.Session()
sess = sagemaker.session.Session(boto_session=boto3_session)
aws_role = sess.get_caller_identity_arn()
sagemaker_session_bucket = sess.default_bucket()
sagemaker_client = boto3.client('sagemaker', region_name=boto3_session.region_name)
print(boto3_session.region_name)

us-west-2


In [7]:
kendra_index_id = '1a1d7832-1658-4cbf-9369-57bacc68f503' ## replace this value with your specific Kendra index id
os.environ['KENDRA_INDEX_ID'] = kendra_index_id
kendra = boto3.client("kendra", boto3_session.region_name)
# Kendra retriever
retriever = AmazonKendraRetriever(
    index_id=kendra_index_id,
    region_name=boto3_session.region_name,
    top_k=5
)

In [8]:
question="Tell me about the return of office policy"
docs = retriever.get_relevant_documents(question)
for doc in docs:
    print(doc.page_content)

Document Title: 2022-Shareholder-Letter.pdf
Document Excerpt: 
free shipping for all online grocery orders over $35) and amended them. We also reprioritized where to spend our resources, which ultimately led to the hard decision to eliminate 27,000 corporate roles. There are a number of other changes that we’ve made over the last several months to streamline our overall costs, and like most leadership teams, we’ll continue to evaluate what we’re seeing in our business and proceed adaptively. We also looked hard at how we were working together as a team and asked our corporate employees to come back to the office at least three days a week, beginning in May. During the pandemic, our employees rallied to get work done from home and did everything possible to keep up with the unexpected circumstances that presented themselves. It was impressive and I’m proud of the way our collective team came together to overcome unprecedented challenges for our customers, communities, and business. But,

In [9]:
model_name = "contextual-answers"
endpoint_name = f'{model_name}-endpoint'

In [18]:
context = docs[0].page_content
print('Context : ', context)

# True context
response = ai21.Answer.execute(
    context=context,
    question=question,
    destination=ai21.SageMakerDestination(endpoint_name)
)
print('AI21 LLM Response:')
print(response.answer)

Context :  Document Title: 2022-Shareholder-Letter.pdf
Document Excerpt: 
free shipping for all online grocery orders over $35) and amended them. We also reprioritized where to spend our resources, which ultimately led to the hard decision to eliminate 27,000 corporate roles. There are a number of other changes that we’ve made over the last several months to streamline our overall costs, and like most leadership teams, we’ll continue to evaluate what we’re seeing in our business and proceed adaptively. We also looked hard at how we were working together as a team and asked our corporate employees to come back to the office at least three days a week, beginning in May. During the pandemic, our employees rallied to get work done from home and did everything possible to keep up with the unexpected circumstances that presented themselves. It was impressive and I’m proud of the way our collective team came together to overcome unprecedented challenges for our customers, communities, and bus

In [20]:
irrelevant_question="""Did COVID-19 eventually help the economy?"""
context = docs[0].page_content
#Irrelevant Question

response = ai21.Answer.execute(
    context=context,
    question=irrelevant_question,
    destination=ai21.SageMakerDestination(endpoint_name)
)
print('AI21 LLM Response:')
print(response.answer)

AI21 LLM Response:
Answer not in document
